# B09705039_HW04

In [1]:
# import module
from nltk.stem import PorterStemmer
import math
import pandas as pd
import numpy as np

In [2]:
# All tokenized doc result
doc_amount = 1095
all_doc = []
for doc in range(1, doc_amount + 1):
    # Read file.
    path = "./data/" + str(doc) + ".txt"
    f = open(path, 'r')
    all_text = f.read()
    f.close()

    # Tokenization.
    # signs that can be ignored: We only listed a few here, if needed more we can add more.
    nonAlphanumeric = [",", "'", ";", ":", '"', "@", "!", "?", "(", ")", "[", "]", "<", ">", "=", "+", "^", "$", "~", "*", "/", "{", "}", "&", "#", "%","`", "_"]
    for i in nonAlphanumeric:
        all_text = all_text.replace(i, " ")

    # periods: concatenate
    all_text = all_text.replace(".", "")
    # hyphens: concatenate
    all_text = all_text.replace("-", "")
    
    # remove digits
    all_text = ''.join([i for i in all_text if not i.isdigit()])

    tokenize = all_text.split()

    # Lowercasing everything.
    lowercase = []
    for i in tokenize:
        lowercase.append(i.lower())
        
    # Stopword removal.
    # Read stopwords file. stopwords.txt is generated by nltk.
    path = 'stopwords.txt'
    f2 = open(path, 'r')
    stop_words = f2.read()
    f2.close()
    
    # Removal start.
    stop_words = stop_words.split()
    stopword_removed = []
    for w in lowercase:
        if w not in stop_words:
            stopword_removed.append(w)
        
    # Stemming using Porter’s algorithm.
    ps = PorterStemmer()

    after_stemming = []
    for w in stopword_removed:
        after_stemming.append(ps.stem(w))
        
    all_doc.append(after_stemming)

In [3]:
# get tf, df dict
tf_dict_arr = []
df_dict = {}
for i in range(doc_amount):
    # count tf
    temp_tf_dict = {}
    for j in all_doc[i]:
        if j not in temp_tf_dict.keys():
            temp_tf_dict[j] = 1
        else:
            temp_tf_dict[j] += 1
    tf_dict_arr.append(temp_tf_dict)
    # count df
    for x in temp_tf_dict.keys():
        if x not in df_dict.keys():
            df_dict[x] = 1
        else:
            df_dict[x] += 1

In [4]:
# Count tf-idf unit vector
# Calculate idf
sortednames = sorted(df_dict.keys(), key=lambda x:x.lower())
idf = {}
for i in sortednames:
    idf[i] = math.log10(doc_amount/df_dict[i])
# Calculate each document's tf-idf unit vector
tf_idf_norm = []
for i in tf_dict_arr:
    tf_idf_temp = {}
    norm = 0
    for j in i.keys():
        tf_idf_temp[sortednames.index(j)] = i[j] * idf[j]
        norm += (tf_idf_temp[sortednames.index(j)]) ** 2
    norm = math.sqrt(norm)
    for j in i.keys():
        tf_idf_temp[sortednames.index(j)] = tf_idf_temp[sortednames.index(j)] / norm
    tf_idf_norm.append(tf_idf_temp)

In [5]:
# Count cosine similarity
def cosine(Docx, Docy):
    cs = 0
    for i in Docx.keys():
        if i in Docy.keys():
            cs += (Docx[i] * Docy[i])
    return cs

In [6]:
# Count similarity matrix
sim_mat = np.zeros((doc_amount, doc_amount))

for i in range(doc_amount):
    for j in range(doc_amount):
        sim_mat[i][j] = cosine(tf_idf_norm[i], tf_idf_norm[j])

In [7]:
print(sim_mat)

[[1.         0.18056711 0.29792948 ... 0.03442438 0.02916595 0.04886198]
 [0.18056711 1.         0.1941756  ... 0.02396501 0.0107888  0.02126824]
 [0.29792948 0.1941756  1.         ... 0.04371176 0.02261194 0.03979733]
 ...
 [0.03442438 0.02396501 0.04371176 ... 1.         0.13198254 0.02126932]
 [0.02916595 0.0107888  0.02261194 ... 0.13198254 1.         0.01944293]
 [0.04886198 0.02126824 0.03979733 ... 0.02126932 0.01944293 1.        ]]


In [8]:
# HAC
import copy

# Initialize
# DeepCopy the similarity matrix above
sim_mat_copy = copy.deepcopy(sim_mat)
I = [1] * doc_amount
A = []

# Calculate
for k in range(doc_amount - 1):
    max_sim = -1
    current_c = -1
    delete_c = -1
    for i in range(doc_amount):
        if I[i] == 0:
            continue
        for j in range(i + 1, doc_amount):
            if I[j] == 0:
                continue
            else:
                if sim_mat_copy[i][j] > max_sim:
                    max_sim = sim_mat_copy[i][j]
                    current_c = i
                    delete_c = j
    A.append([current_c, delete_c])
    for l in range(doc_amount):
        if (l != current_c) and (l != delete_c) and (I[l] == 1):
            sim_mat_copy[current_c][l] = min(sim_mat_copy[current_c][l], sim_mat_copy[delete_c][l])
            sim_mat_copy[l][current_c] = min(sim_mat_copy[l][current_c], sim_mat_copy[l][delete_c])
    I[delete_c] = 0

In [9]:
print(sim_mat_copy)

[[1.         0.10247379 0.02105214 ... 0.01094097 0.02007929 0.01150417]
 [0.10247379 1.         0.05900125 ... 0.02396501 0.0107888  0.02126824]
 [0.02105214 0.05900125 1.         ... 0.01332079 0.00842161 0.02074804]
 ...
 [0.01094097 0.02396501 0.01332079 ... 1.         0.13198254 0.02126932]
 [0.02007929 0.0107888  0.00842161 ... 0.13198254 1.         0.01944293]
 [0.01150417 0.02126824 0.02074804 ... 0.02126932 0.01944293 1.        ]]


In [10]:
print(A)

[[925, 927], [7, 8], [526, 528], [942, 943], [47, 48], [620, 621], [661, 662], [194, 228], [194, 229], [563, 564], [563, 594], [563, 595], [704, 705], [475, 476], [731, 732], [304, 308], [100, 105], [791, 795], [210, 211], [210, 212], [847, 848], [190, 191], [1033, 1035], [499, 500], [154, 157], [831, 837], [506, 507], [582, 583], [675, 676], [242, 243], [1084, 1087], [680, 683], [329, 332], [197, 199], [821, 823], [854, 855], [895, 896], [99, 101], [196, 201], [604, 606], [519, 522], [824, 827], [735, 736], [490, 492], [898, 905], [498, 499], [840, 841], [371, 380], [838, 839], [968, 969], [53, 54], [87, 88], [910, 911], [545, 546], [820, 821], [953, 954], [999, 1007], [230, 234], [108, 109], [884, 885], [995, 996], [93, 97], [55, 56], [129, 130], [967, 968], [244, 249], [1083, 1084], [718, 721], [326, 327], [530, 531], [501, 502], [196, 198], [301, 312], [838, 840], [454, 455], [967, 970], [614, 615], [1040, 1041], [33, 34], [280, 281], [597, 601], [921, 923], [573, 574], [506, 511],

In [11]:
# Get cluster from A
def get_cluster(length):
    cluster_arr = []
    a = copy.deepcopy(A)
    for i in range(doc_amount):
        cluster_arr.append([i])

    for j in a:
        for i in range(len(cluster_arr)):
            if cluster_arr[i][0] == j[0]:
                for k in range(len(cluster_arr)):
                    if cluster_arr[k][0] == j[1]:
                        cluster_arr[i] = cluster_arr[i] + cluster_arr[k]
                        cluster_arr.pop(k)
                        break
                break
        if len(cluster_arr) == length:
            return cluster_arr

In [12]:
# Output 8.txt, 13.txt, 20.txt
def write_to_file(num, cluster):
    path = str(num) + '.txt'
    f = open(path, 'w')

    writeArr = ""
    for i in cluster:
        i.sort()
        for j in i:
            writeArr += str(j + 1) + "\n"
        writeArr += "\n"

    f.writelines(writeArr)
    f.close()

In [13]:
# Cluster for 8 groups
cluster = get_cluster(8)
print(len(cluster))
write_to_file(8, cluster)

8


In [14]:
# Cluster for 13 groups
cluster = get_cluster(13)
print(len(cluster))
write_to_file(13, cluster)

13


In [15]:
# Cluster for 20 groups
cluster = get_cluster(20)
print(len(cluster))
write_to_file(20, cluster)

20
